In [0]:
%skip
checkpoint = spark.sql("""DESCRIBE EXTERNAL LOCATION checkpoints""").select("url").collect()[0].url
landing = spark.sql("""DESCRIBE EXTERNAL LOCATION landing""").select("url").collect()[0].url
silver = spark.sql("""DESCRIBE EXTERNAL LOCATION silver""").select("url").collect()[0].url

In [0]:
%run "/Workspace/Dev/04. Common"

In [0]:
# print(checkpoint)
# abfss://checkpoints@adlsstorage44.dfs.core.windows.net/SilverRoadsLoad/Checkpt

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")

environment = env

In [0]:
def read_BronzeRoadsTable(environment):
    print('Reading the Bronze Table raw_roads Data : ',end='')
    df_bronzeRoads = (spark.readStream
                      .table(f"`{environment}_catalog`.`bronze`.`raw_roads`"))
    
    print(f'Reading {environment}_catalog.bronze.raw_roads Success!')
    print("**********************************")
    return df_bronzeRoads

In [0]:
def road_Category(df):
    print("Creating column Road_Category_Name: ",end="")
    from pyspark.sql.functions import when,col
    
    df_road_cat = df.withColumn("Road_Category_Name",
                  when(col("Road_Category")=='TA','Class A Trunk Road')
                    .when(col("Road_Category")=='TM','Class A Trunk Motorway')
                    .when(col("Road_Category")=='PA','Class A Principal Road')
                    .when(col("Road_Category")=='PM','Class A Prinicipal Motorway')
                    .when(col("Road_Category")=='M','Class B Road')
                    .otherwise('NA')
                       )
    print('Success!! ')
    print('***********************')
    return df_road_cat



In [0]:
def road_Type(df):
    print("Creating column Road_Type : ",end="")
    from pyspark.sql.functions import when, col
    df_road_type = df.withColumn("Road_Type",
                                 when(col("Road_Category_Name").like('%Class A%'),'Major')
                                 .when(col("Road_Category_Name").like('%Class B%'),'Minor')
                                 .otherwise("NA")
                                 )
    print('Success!! ')
    print('***********************')
    return df_road_type

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_Roads=(StreamingDF.writeStream
                              .format('delta')
                              .option('checkpointLocation',checkpoint+ "SilverRoadsLoad/Checkpt_v2")
                              .outputMode('append')
                              .queryName("SilverRoadWriteStream")
                              .trigger(availableNow=True)
                              .toTable(f"`{environment}_catalog`.`silver`.`silver_roads`")                             
                              )
        
    print(f'Writing `{environment}_catalog`.`silver`.`silver_roads` Success!')


In [0]:
## Read Bronze Data
df_bronzeRoads = read_BronzeRoadsTable(env)

## Remove Duplicates
df_dups = remove_duplicates(df_bronzeRoads)

# To raplce any NULL values
Allcolumns =df_dups.schema.names
df_nulls = handle_nulls(df_dups,Allcolumns)

## Creating Road_Category_name 
df_roadCat = road_Category(df_nulls)

## Creating Road_Type column
df_type = road_Type(df_roadCat)

## Writing data to silver_roads table

write_Roads_SilverTable(df_type,env)

In [0]:
# spark.sql("""Select count(1) from `dev_catalog`.`silver`.`silver_roads`""").display()